Pull data from the `nps-public-data` [BigQuery project](https://github.com/tonymet/nps-public-data), load to DuckDB tables and export to parquet.

In [1]:
import duckdb
from google.cloud import bigquery

con = duckdb.connect("../dbt/nps_data/nps_data.db")

bqclient = bigquery.Client(project="nps-public-data")

dataset_ref = f"nps-public-data.nps_public_data"

/Users/matt/.pyenv/versions/3.10.0/envs/nps-dbt/lib/python3.10/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [2]:
schema = "raw_nps_api"

con.sql(f"CREATE SCHEMA IF NOT EXISTS {schema}")

select_table = ['parks', 'campgrounds', 'parkinglots', 'webcams', 'visitorcenters']

tables = [table for table in bqclient.list_tables(dataset_ref) if table.table_id in select_table]

for table in tables:

    table_str = f"{table.project}.{table.dataset_id}.{table.table_id}"

    table = bigquery.TableReference.from_string(table_str)

    rows = bqclient.list_rows(table)

    # this is a terrible pun
    arr_rows = rows.to_arrow(create_bqstorage_client=True)

    sql_ref = f"{schema}.{table.table_id}"

    con.sql(
        f"""
        DROP TABLE IF EXISTS {sql_ref};
        CREATE TABLE {sql_ref} AS SELECT * FROM arr_rows;
        """
    )

    print(table_str)

con.sql("SHOW ALL tables")

nps-public-data.nps_public_data.campgrounds
nps-public-data.nps_public_data.parkinglots
nps-public-data.nps_public_data.visitorcenters
nps-public-data.nps_public_data.webcams


┌──────────┬────────────────────┬────────────────────┬──────────────────────┬──────────────────────────────┬───────────┐
│ database │       schema       │        name        │     column_names     │         column_types         │ temporary │
│ varchar  │      varchar       │      varchar       │      varchar[]       │          varchar[]           │  boolean  │
├──────────┼────────────────────┼────────────────────┼──────────────────────┼──────────────────────────────┼───────────┤
│ nps_data │ analysis           │ calendar           │ [date_day, date_we…  │ [TIMESTAMP, DATE, DATE, DA…  │ false     │
│ nps_data │ main_seed_data     │ nps_irma_1979_2023 │ [ParkName, UnitCod…  │ [VARCHAR, VARCHAR, VARCHAR…  │ false     │
│ nps_data │ nps_data           │ nps_irma_1979_2023 │ [ParkName, UnitCod…  │ [VARCHAR, VARCHAR, VARCHAR…  │ false     │
│ nps_data │ raw_nps_api        │ campgrounds        │ [lastIndexedDate, …  │ [VARCHAR, DOUBLE, STRUCT(c…  │ false     │
│ nps_data │ raw_nps_api        